In [ ]:
import pandas as pd

# CSV 불러오기
posts = pd.read_csv("data/cafe_posts_clean.csv")     

# 50개 행을 랜덤 추출 (랜덤 시드 고정하면 재현 가능)
sample_posts = posts.sample(n=50, random_state=42)

# 결과 확인
print(sample_posts.shape)
print(sample_posts.head())

# 필요하다면 파일로 저장
sample_posts.to_csv("data/cafe_posts_sample50.csv", index=False, encoding="utf-8-sig")
print("✅ 50개 샘플 저장 완료")

핸들 샘플링 과정
1. 매장명 동의어 -> 대표성 텍스트로 분류하여 추출 
    : 매장명 리스트 [사전]필요. jewerly_place_with_brandname
2. 매장명이 언급된 게시글의 번호, 브랜드, 감정분류 
    columns : article_id, brand_name, sentiment
    : 감정분류 [사전] 필요. place_sentiment_keywords


In [ ]:
import pandas as pd
import re
import json

# -----------------------------
# 공통 유틸
# -----------------------------
def find_column(df: pd.DataFrame, candidates: list[str]) -> str | None:
    """대소문자/공백 무시하고 후보 중 존재하는 첫 컬럼명을 찾아 반환"""
    norm = {c.lower().strip(): c for c in df.columns}
    for cand in candidates:
        key = cand.lower().strip()
        if key in norm:
            return norm[key]
    return None

def safe_text(row: pd.Series, col: str | None) -> str:
    """Series에서 안전하게 텍스트 추출 (없으면 빈 문자열)"""
    if not col:
        return ""
    # pandas Series.get 사용 → 없으면 기본값 반환
    return str(row.get(col, "") or "")

# -----------------------------
# 1) 데이터 불러오기 + 샘플링
# -----------------------------
def load_and_sample(filepath: str, n=50, random_state=42) -> pd.DataFrame:
    df = pd.read_csv(filepath)
    sample_df = df.sample(n=n, random_state=random_state)
    return sample_df.reset_index(drop=True)

# -----------------------------
# 2) 매장명 정규화
# -----------------------------
def load_place_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def normalize_place(text: str, place_dict: dict) -> str | None:
    # 동의어는 정규식 특수문자 이스케이프 + 대소문자 무시
    for standard, synonyms in place_dict.items():
        for syn in synonyms:
            if syn and re.search(re.escape(syn), text, flags=re.IGNORECASE):
                return standard
    return None

# -----------------------------
# 3) 감정 분류 (카테고리별)
# -----------------------------
def load_sentiment_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def classify_sentiment(text: str, sentiment_dict: dict) -> str:
    # sentiment_dict의 key가 "인테리어_긍정" 같은 카테고리라고 가정
    for category, keywords in sentiment_dict.items():
        if any(kw and kw in text for kw in keywords):
            return category
    return "기타_중립"  # 매칭 없으면 기본 라벨

# -----------------------------
# 4) 실행 부분
# -----------------------------
def main():
    # ✅ 파일 경로: JSON 확장자 확인!
    CLEAN_FILE = "data/cafe_posts_clean.csv"
    PLACE_DICT_FILE = "data/jewelry_place_dict.json"
    SENTIMENT_DICT_FILE = "data/place_sentiment_keywords_by_category.json"

    # 1) 샘플링
    df = load_and_sample(CLEAN_FILE, n=50, random_state=42)

    # 2) 컬럼 자동 감지
    naver_article_id_col = find_column(df, ["naver_article_id", "post_id", "id", "글번호", "번호", "no"])
    title_col      = find_column(df, ["title", "제목"])
    text_col       = find_column(df, ["text", "본문", "내용"])
    comments_col   = find_column(df, ["comments", "comment", "댓글", "코멘트"])

    # 참고용: 어떤 컬럼이 매핑됐는지 출력
    print("[Column mapping]")
    print(" naver_article_id_col:", naver_article_id_col)
    print(" title_col     :", title_col)
    print(" text_col      :", text_col)
    print(" comments_col  :", comments_col)

    # 3) 사전 로드
    place_dict     = load_place_dict(PLACE_DICT_FILE)
    sentiment_dict = load_sentiment_dict(SENTIMENT_DICT_FILE)

    # 4) 라벨링
    results = []
    for idx, row in df.iterrows():
        # 텍스트 구성: 없으면 빈 문자열로 안전 처리
        text = " ".join([
            safe_text(row, title_col),
            safe_text(row, text_col),
            safe_text(row, comments_col),
        ])

        place = normalize_place(text, place_dict)
        category_sentiment = classify_sentiment(text, sentiment_dict)

        if place:  # 매장 언급된 경우만
            # ✅ article_id가 없으면 인덱스로 대체
            article_id = row.get(naver_article_id_col, idx) if naver_article_id_col else idx
            results.append({
                "article_id": article_id,
                # 원래 요구 컬럼명이 brand_name이었지만, 실제로는 '매장명'이 들어가므로 혼동이면 'place_name'으로 쓰는 걸 권장
                "brand_name": place,  # 필요 시 "place_name": place 로 변경 권장
                "sentiment": category_sentiment
            })

    results_df = pd.DataFrame(results)
    print("\n=== 라벨링 결과 미리보기 (gkdnl 10행) ===")
    print(results_df.tail(10))

if __name__ == "__main__":
    main()


하이브리드 방식: LLM+dict / 사전에 있으면 그대로 출력, 없으면 LLM 으로 분류해서 출력.

In [ ]:

import pandas as pd
import os
import re
import json
from dotenv import load_dotenv
import google.generativeai as genai  # ✅ Gemini SDK

# ------------------------
# ① 환경설정 & API 키 불러오기
# ------------------------
load_dotenv()
google_key = os.getenv("GOOGLE_API_KEY")
if not google_key:
    raise ValueError("환경변수 GOOGLE_API_KEY가 없습니다. .env 파일 확인!")

# ------------------------
# ② Gemini 클라이언트 초기화
# ------------------------
genai.configure(api_key=google_key)
gemini_model = genai.GenerativeModel("gemini-1.5-flash")



# -----------------------------
# 1) 데이터 불러오기 + 샘플링
# -----------------------------
def load_and_sample(filepath: str, n=50, random_state=42) -> pd.DataFrame:
    df = pd.read_csv(filepath)
    sample_df = df.sample(n=n, random_state=random_state)
    return sample_df.reset_index(drop=True)

# -----------------------------
# 2) 매장명 정규화 (Rule 기반)
# -----------------------------
def load_place_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def normalize_place_rule(text: str, place_dict: dict) -> str | None:
    for standard, synonyms in place_dict.items():
        for syn in synonyms:
            if syn and re.search(re.escape(syn), text, flags=re.IGNORECASE):
                return standard
    return None

# -----------------------------
# 3) 감정 분류 (Rule 기반)
# -----------------------------
def load_sentiment_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def classify_sentiment_rule(text: str, sentiment_dict: dict) -> str | None:
    for category, keywords in sentiment_dict.items():
        if any(kw in text for kw in keywords):
            return category
    return None

# -----------------------------
# 4) LLM 보완
# -----------------------------
def classify_with_llm(text: str, place_dict: dict, sentiment_categories: list[str]) -> tuple[str, str]:
    prompt = f"""
    다음 텍스트에서 언급된 매장명을 표준 매장명으로 추출하고,
    감정은 아래 카테고리 중 가장 적합한 하나를 선택해줘.

    텍스트: {text}

    매장 후보 (예시): {list(place_dict.keys())[:20]} ...  # 전체가 너무 많으면 일부만 예시 제공

    감정 카테고리 후보:
    {sentiment_categories}

    출력 형식:
    매장명: <표준매장명 또는 None>
    감정: <카테고리>
    출력은 반드시 JSON 형식으로만 해.
    예: {"매장명": "현대백화점 무역센터점", "감정": "인테리어_긍정"}
    """

    response = gemini_model.generate_content(prompt)
    answer = response.text.strip()

    place, sentiment = None, "기타_중립"
    for line in answer.splitlines():
        if line.startswith("매장명:"):
            place = line.replace("매장명:", "").strip()
        if line.startswith("감정:"):
            sentiment = line.replace("감정:", "").strip()

    return place, sentiment


# -----------------------------
# 5) 실행 부분 (Hybrid 적용)
# -----------------------------
def main():
    CLEAN_FILE = "data/cafe_posts_clean.csv"
    PLACE_DICT_FILE = "data/jewelry_place_dict.json"
    SENTIMENT_DICT_FILE = "data/place_sentiment_keywords_by_category.json"

    df = load_and_sample(CLEAN_FILE, n=50, random_state=42)
    place_dict = load_place_dict(PLACE_DICT_FILE)
    sentiment_dict = load_sentiment_dict(SENTIMENT_DICT_FILE)
    sentiment_categories = list(sentiment_dict.keys())

    results = []
    for idx, row in df.iterrows():
        text = f"{row.get('title','')} {row.get('text','')} {row.get('comments','')}"

        # 1) Rule 기반 먼저 시도
        place = normalize_place_rule(text, place_dict)
        sentiment = classify_sentiment_rule(text, sentiment_dict)

        # 2) Rule 실패 시 LLM 호출
        if not place or not sentiment:
            place_llm, sentiment_llm = classify_with_llm(text, place_dict, sentiment_categories)
            place = place or place_llm
            sentiment = sentiment or sentiment_llm

        if place:
            results.append({
                "article_id": row.get("article_id", idx),
                "place_name": place,
                "sentiment": sentiment
            })

    results_df = pd.DataFrame(results)
    print("\n=== 라벨링 결과 미리보기 (상위 10행) ===")
    print(results_df.head(10))

if __name__ == "__main__":
    main()


게시글번호 naver_article_id 반영되도록.

In [ ]:
import pandas as pd
import os
import re
import json
from dotenv import load_dotenv
import google.generativeai as genai  # Gemini SDK

# ------------------------
# ① 환경설정 & API 키 불러오기
# ------------------------
load_dotenv()
google_key = os.getenv("GOOGLE_API_KEY")
if not google_key:
    raise ValueError("환경변수 GOOGLE_API_KEY가 없습니다. .env 파일 확인!")

# ------------------------
# ② Gemini 클라이언트 초기화
# ------------------------
genai.configure(api_key=google_key)
gemini_model = genai.GenerativeModel("gemini-1.5-flash")

# -----------------------------
# 1) 데이터 불러오기 + 샘플링
# -----------------------------
def load_and_sample(filepath: str, n=50, random_state=42) -> pd.DataFrame:
    df = pd.read_csv(filepath)
    sample_df = df.sample(n=n, random_state=random_state)
    return sample_df.reset_index(drop=True)

# -----------------------------
# 2) 매장명 정규화 (Rule 기반)
# -----------------------------
def load_place_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def normalize_place_rule(text: str, place_dict: dict) -> str | None:
    for standard, synonyms in place_dict.items():
        for syn in synonyms:
            if syn and re.search(re.escape(syn), text, flags=re.IGNORECASE):
                return standard
    return None

# -----------------------------
# 3) 감정 분류 (Rule 기반)
# -----------------------------
def load_sentiment_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def classify_sentiment_rule(text: str, sentiment_dict: dict) -> str | None:
    for category, keywords in sentiment_dict.items():
        if any(kw in text for kw in keywords):
            return category
    return None

# -----------------------------
# 4) LLM 보완 (Gemini)
# -----------------------------
def classify_with_llm(text: str, place_dict: dict, sentiment_categories: list[str]) -> tuple[str, str]:
    prompt = f"""
    다음 텍스트에서 언급된 매장명을 표준 매장명으로 추출하고,
    감정은 아래 카테고리 중 가장 적합한 하나를 선택해줘.

    텍스트: {text}

    매장 후보 (예시): {list(place_dict.keys())[:20]}

    감정 카테고리 후보:
    {sentiment_categories}

    출력 형식:
    매장명: <표준매장명 또는 None>
    감정: <카테고리>
    """

    response = gemini_model.generate_content(prompt)
    answer = response.text.strip()

    place, sentiment = None, "기타_중립"
    for line in answer.splitlines():
        if line.startswith("매장명:"):
            place = line.replace("매장명:", "").strip()
        if line.startswith("감정:"):
            sentiment = line.replace("감정:", "").strip()

    return place, sentiment

# -----------------------------
# 5) 실행 부분 (Hybrid 적용)
# -----------------------------
def main():
    CLEAN_FILE = "data/cafe_posts_clean.csv"
    PLACE_DICT_FILE = "data/jewelry_place_dict.json"
    SENTIMENT_DICT_FILE = "data/place_sentiment_keywords_by_category.json"

    df = load_and_sample(CLEAN_FILE, n=50, random_state=42)
    place_dict = load_place_dict(PLACE_DICT_FILE)
    sentiment_dict = load_sentiment_dict(SENTIMENT_DICT_FILE)
    sentiment_categories = list(sentiment_dict.keys())

    results = []
    for _, row in df.iterrows():
        text = f"{row.get('title','')} {row.get('text','')} {row.get('comments','')}"

        # 1) Rule 기반 먼저 시도
        place = normalize_place_rule(text, place_dict)
        sentiment = classify_sentiment_rule(text, sentiment_dict)

        # 2) Rule 실패 시 LLM 호출
        if not place or not sentiment:
            place_llm, sentiment_llm = classify_with_llm(text, place_dict, sentiment_categories)
            place = place or place_llm
            sentiment = sentiment or sentiment_llm

        if place:
            results.append({
                "article_id": row.get("naver_article_id", None),  # ✅ 네이버 게시글 번호
                "place_name": place,
                "sentiment": sentiment
            })

    results_df = pd.DataFrame(results)
    print("\n=== 라벨링 결과 미리보기 (상위 10행) ===")
    print(results_df.head(10))

if __name__ == "__main__":
    main()


pbrand_name, location_lv1, location_lv2 정보도 구분할 수 있게 dic.json

In [ ]:
import pandas as pd
import os
import re
import json
from dotenv import load_dotenv
import google.generativeai as genai  # Gemini SDK

# ------------------------
# ① 환경설정 & API 키 불러오기
# ------------------------
load_dotenv()
google_key = os.getenv("GOOGLE_API_KEY")
if not google_key:
    raise ValueError("환경변수 GOOGLE_API_KEY가 없습니다. .env 파일 확인!")

# ------------------------
# ② Gemini 클라이언트 초기화
# ------------------------
genai.configure(api_key=google_key)
gemini_model = genai.GenerativeModel("gemini-1.5-flash")

# -----------------------------
# 1) 데이터 불러오기 + 샘플링
# -----------------------------
def load_and_sample(filepath: str, n=50, random_state=42) -> pd.DataFrame:
    df = pd.read_csv(filepath)
    sample_df = df.sample(n=n, random_state=random_state)
    return sample_df.reset_index(drop=True)

# -----------------------------
# 2) 매장명 정규화 (Rule 기반)
# -----------------------------
def load_place_info_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def normalize_place_rule(text: str, place_info_dict: dict) -> str | None:
    for standard, info in place_info_dict.items():
        for syn in info["synonyms"]:
            if syn and re.search(re.escape(syn), text, flags=re.IGNORECASE):
                return standard
    return None

# -----------------------------
# 3) 감정 분류 (Rule 기반)
# -----------------------------
def load_sentiment_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def classify_sentiment_rule(text: str, sentiment_dict: dict) -> str | None:
    for category, keywords in sentiment_dict.items():
        if any(kw in text for kw in keywords):
            return category
    return None

# -----------------------------
# 4) LLM 보완 (Gemini)
# -----------------------------
def classify_with_llm(text: str, place_info_dict: dict, sentiment_categories: list[str]) -> tuple[str, str]:
    prompt = f"""
    다음 텍스트에서 언급된 매장명을 표준 매장명으로 추출하고,
    감정은 아래 카테고리 중 가장 적합한 하나를 선택해줘.

    텍스트: {text}

    매장 후보 (예시): {list(place_info_dict.keys())[:20]}

    감정 카테고리 후보:
    {sentiment_categories}

    출력 형식:
    매장명: <표준매장명 또는 None>
    감정: <카테고리>
    """

    response = gemini_model.generate_content(prompt)
    answer = response.text.strip()

    place, sentiment = None, "기타_중립"
    for line in answer.splitlines():
        if line.startswith("매장명:"):
            place = line.replace("매장명:", "").strip()
        if line.startswith("감정:"):
            sentiment = line.replace("감정:", "").strip()

    return place, sentiment

# -----------------------------
# 5) 실행 부분 (Hybrid 적용)
# -----------------------------
def main():
    CLEAN_FILE = "data/cafe_posts_clean.csv"
    PLACE_INFO_FILE = "data/jewelry_place_info_dict.json"   # ✅ 새로운 JSON
    SENTIMENT_DICT_FILE = "data/place_sentiment_keywords_by_category.json"

    df = load_and_sample(CLEAN_FILE, n=50, random_state=42)
    place_info_dict = load_place_info_dict(PLACE_INFO_FILE)
    sentiment_dict = load_sentiment_dict(SENTIMENT_DICT_FILE)
    sentiment_categories = list(sentiment_dict.keys())

    results = []
    for _, row in df.iterrows():
        text = f"{row.get('title','')} {row.get('text','')} {row.get('comments','')}"

        # 1) Rule 기반 먼저 시도
        place = normalize_place_rule(text, place_info_dict)
        sentiment = classify_sentiment_rule(text, sentiment_dict)

        # 2) Rule 실패 시 LLM 호출
        if not place or not sentiment:
            place_llm, sentiment_llm = classify_with_llm(text, place_info_dict, sentiment_categories)
            place = place or place_llm
            sentiment = sentiment or sentiment_llm

        if place:
            info = place_info_dict.get(place, {})
            results.append({
                "article_id": row.get("naver_article_id", None),
                "place_name": place,
                "pbrand_name": info.get("pbrand_name"),
                "location_lv1": info.get("location_lv1"),
                "location_lv2": info.get("location_lv2"),
                "sentiment": sentiment
            })

    results_df = pd.DataFrame(results)
    print("\n=== 라벨링 결과 미리보기 (상위 10행) ===")
    print(results_df.head(10))

if __name__ == "__main__":
    main()


조건 필터 추가: place_name, pbrand_name, location_lv1, location_lv2 모두 None/"None"이면 결과에서 제외

In [ ]:
import pandas as pd
import os
import re
import json
from dotenv import load_dotenv
import google.generativeai as genai  #  Gemini SDK

# ------------------------
# ① 환경설정 & API 키 불러오기
# ------------------------
load_dotenv()
google_key = os.getenv("GOOGLE_API_KEY")
if not google_key:
    raise ValueError("환경변수 GOOGLE_API_KEY가 없습니다. .env 파일 확인!")

# ------------------------
# ② Gemini 클라이언트 초기화
# ------------------------
genai.configure(api_key=google_key)
gemini_model = genai.GenerativeModel("gemini-1.5-flash")

# -----------------------------
# 1) 데이터 불러오기 + 샘플링
# -----------------------------
def load_and_sample(filepath: str, n=50, random_state=42) -> pd.DataFrame:
    df = pd.read_csv(filepath)
    sample_df = df.sample(n=n, random_state=random_state)
    return sample_df.reset_index(drop=True)

# -----------------------------
# 2) 매장명 정규화 (Rule 기반)
# -----------------------------
def load_place_info_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def normalize_place_rule(text: str, place_info_dict: dict) -> str | None:
    for standard, info in place_info_dict.items():
        for syn in info["synonyms"]:
            if syn and re.search(re.escape(syn), text, flags=re.IGNORECASE):
                return standard
    return None

# -----------------------------
# 3) 감정 분류 (Rule 기반)
# -----------------------------
def load_sentiment_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def classify_sentiment_rule(text: str, sentiment_dict: dict) -> str | None:
    for category, keywords in sentiment_dict.items():
        if any(kw in text for kw in keywords):
            return category
    return None

# -----------------------------
# 4) LLM 보완 (Gemini)
# -----------------------------
def classify_with_llm(text: str, place_info_dict: dict, sentiment_categories: list[str]) -> tuple[str, str]:
    prompt = f"""
    다음 텍스트에서 언급된 매장명을 표준 매장명으로 추출하고,
    감정은 아래 카테고리 중 가장 적합한 하나를 선택해줘.

    텍스트: {text}

    매장 후보 (예시): {list(place_info_dict.keys())[:20]}

    감정 카테고리 후보:
    {sentiment_categories}

    출력 형식:
    매장명: <표준매장명 또는 None>
    감정: <카테고리>
    """

    response = gemini_model.generate_content(prompt)
    answer = response.text.strip()

    place, sentiment = None, "기타_중립"
    for line in answer.splitlines():
        if line.startswith("매장명:"):
            place = line.replace("매장명:", "").strip()
        if line.startswith("감정:"):
            sentiment = line.replace("감정:", "").strip()

    return place, sentiment

# -----------------------------
# 5) 실행 부분 (Hybrid 적용)
# -----------------------------
def main():
    CLEAN_FILE = "data/cafe_posts_clean.csv"
    PLACE_INFO_FILE = "data/jewelry_place_info_dict.json"   # 새로운 JSON
    SENTIMENT_DICT_FILE = "data/place_sentiment_keywords_by_category.json"

    df = load_and_sample(CLEAN_FILE, n=50, random_state=42)
    place_info_dict = load_place_info_dict(PLACE_INFO_FILE)
    sentiment_dict = load_sentiment_dict(SENTIMENT_DICT_FILE)
    sentiment_categories = list(sentiment_dict.keys())

    results = []
    for _, row in df.iterrows():
        text = f"{row.get('title','')} {row.get('text','')} {row.get('comments','')}"

        # 1) Rule 기반 먼저 시도
        place = normalize_place_rule(text, place_info_dict)
        sentiment = classify_sentiment_rule(text, sentiment_dict)

        # 2) Rule 실패 시 LLM 호출
        if not place or not sentiment:
            place_llm, sentiment_llm = classify_with_llm(text, place_info_dict, sentiment_categories)
            place = place or place_llm
            sentiment = sentiment or sentiment_llm

        if place:
            info = place_info_dict.get(place, {})
            record = {
                "article_id": row.get("naver_article_id", None),
                "place_name": place,
                "pbrand_name": info.get("pbrand_name"),
                "location_lv1": info.get("location_lv1"),
                "location_lv2": info.get("location_lv2"),
                "sentiment": sentiment
            }

            # ✅ 모두 None 또는 "None"이면 제외
            if not all((record.get(k) is None or record.get(k) == "None") for k in ["place_name", "pbrand_name", "location_lv1", "location_lv2"]):
                results.append(record)

    results_df = pd.DataFrame(results)
    print("\n=== 라벨링 결과 미리보기 (상위 10행) ===")
    print(results_df.head(10))

if __name__ == "__main__":
    main()


sentiment에서 친절에 관한 레코드만 출력되도록.

In [ ]:
import pandas as pd
import os
import re
import json
from dotenv import load_dotenv
import google.generativeai as genai  # Gemini SDK

# ------------------------
# ① 환경설정 & API 키 불러오기
# ------------------------
load_dotenv()
google_key = os.getenv("GOOGLE_API_KEY")
if not google_key:
    raise ValueError("환경변수 GOOGLE_API_KEY가 없습니다. .env 파일 확인!")

# ------------------------
# ② Gemini 클라이언트 초기화
# ------------------------
genai.configure(api_key=google_key)
gemini_model = genai.GenerativeModel("gemini-1.5-flash")

# -----------------------------
# 1) 데이터 불러오기 + 샘플링
# -----------------------------
def load_and_sample(filepath: str, n=50, random_state=42) -> pd.DataFrame:
    df = pd.read_csv(filepath)
    sample_df = df.sample(n=n, random_state=random_state)
    return sample_df.reset_index(drop=True)

# -----------------------------
# 2) 매장명 정규화 (Rule 기반)
# -----------------------------
def load_place_info_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def normalize_place_rule(text: str, place_info_dict: dict) -> str | None:
    for standard, info in place_info_dict.items():
        for syn in info["synonyms"]:
            if syn and re.search(re.escape(syn), text, flags=re.IGNORECASE):
                return standard
    return None

# -----------------------------
# 3) 감정 분류 (Rule 기반)
# -----------------------------
def load_sentiment_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def classify_sentiment_rule(text: str, sentiment_dict: dict) -> str | None:
    for category, keywords in sentiment_dict.items():
        if any(kw in text for kw in keywords):
            return category
    return None

# -----------------------------
# 4) LLM 보완 (Gemini)
# -----------------------------
def classify_with_llm(text: str, place_info_dict: dict, sentiment_categories: list[str]) -> tuple[str, str]:
    prompt = f"""
    다음 텍스트에서 언급된 매장명을 표준 매장명으로 추출하고,
    감정은 아래 카테고리 중 가장 적합한 하나를 선택해줘.

    텍스트: {text}

    매장 후보 (예시): {list(place_info_dict.keys())[:20]}

    감정 카테고리 후보:
    {sentiment_categories}

    출력 형식:
    매장명: <표준매장명 또는 None>
    감정: <카테고리>
    """

    response = gemini_model.generate_content(prompt)
    answer = response.text.strip()

    place, sentiment = None, "기타_중립"
    for line in answer.splitlines():
        if line.startswith("매장명:"):
            place = line.replace("매장명:", "").strip()
        if line.startswith("감정:"):
            sentiment = line.replace("감정:", "").strip()

    return place, sentiment

# -----------------------------
# 5) 실행 부분 (Hybrid 적용)
# -----------------------------
def main():
    CLEAN_FILE = "data/cafe_posts_clean.csv"
    PLACE_INFO_FILE = "data/jewelry_place_info_dict.json"   # 새로운 JSON
    SENTIMENT_DICT_FILE = "data/place_sentiment_keywords_by_category.json"

    df = load_and_sample(CLEAN_FILE, n=50, random_state=42)
    place_info_dict = load_place_info_dict(PLACE_INFO_FILE)
    sentiment_dict = load_sentiment_dict(SENTIMENT_DICT_FILE)
    sentiment_categories = list(sentiment_dict.keys())

    results = []
    for _, row in df.iterrows():
        text = f"{row.get('title','')} {row.get('text','')} {row.get('comments','')}"

        # 1) Rule 기반 먼저 시도
        place = normalize_place_rule(text, place_info_dict)
        sentiment = classify_sentiment_rule(text, sentiment_dict)

        # 2) Rule 실패 시 LLM 호출
        if not place or not sentiment:
            place_llm, sentiment_llm = classify_with_llm(text, place_info_dict, sentiment_categories)
            place = place or place_llm
            sentiment = sentiment or sentiment_llm

        if place:
            info = place_info_dict.get(place, {})
            record = {
                "article_id": row.get("naver_article_id", None),
                "place_name": place,
                "pbrand_name": info.get("pbrand_name"),
                "location_lv1": info.get("location_lv1"),
                "location_lv2": info.get("location_lv2"),
                "sentiment": sentiment
            }

            # 모두 None 또는 "None"이면 제외
            if not all((record.get(k) is None or record.get(k) == "None") for k in ["place_name", "pbrand_name", "location_lv1", "location_lv2"]):
                results.append(record)
                

    
    results_df = pd.DataFrame(results)
    print("\n=== 라벨링 결과 미리보기 (상위 10행) ===")
    print(results_df.head(10))
    
    # # ✅ 친절 관련 감정만 필터링 
    # results_df = results_df[results_df["sentiment"].str.contains("친절", na=False)]
    # 이렇게 하면 바로 위에서 지정한 상위 10행만 저장됨. (친절 필터링 없이 전체 sentiment)

if __name__ == "__main__":
    main()


sentiment 필터링 없이 전체 CSV로 저장 

In [ ]:
import pandas as pd
import os
import re
import json
from dotenv import load_dotenv
import google.generativeai as genai  # Gemini SDK

# ------------------------
# ① 환경설정 & API 키 불러오기
# ------------------------
load_dotenv()
google_key = os.getenv("GOOGLE_API_KEY")
if not google_key:
    raise ValueError("환경변수 GOOGLE_API_KEY가 없습니다. .env 파일 확인!")

# ------------------------
# ② Gemini 클라이언트 초기화
# ------------------------
genai.configure(api_key=google_key)
gemini_model = genai.GenerativeModel("gemini-1.5-flash")

# ------------------------
# ✅ CSV 저장 헬퍼 함수
# ------------------------
def save_csv(df: pd.DataFrame, path: str):
    """DataFrame을 안전하게 CSV로 저장"""
    df.to_csv(path, index=False, encoding="utf-8-sig")
    print(f"✅ CSV 저장 완료: {path}")

# -----------------------------
# 1) 데이터 불러오기 + 샘플링
# -----------------------------
def load_and_sample(filepath: str, n=50, random_state=42) -> pd.DataFrame:
    df = pd.read_csv(filepath)
    sample_df = df.sample(n=n, random_state=random_state)
    return sample_df.reset_index(drop=True)

# -----------------------------
# 2) 매장명 정규화 (Rule 기반)
# -----------------------------
def load_place_info_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def normalize_place_rule(text: str, place_info_dict: dict) -> str | None:
    for standard, info in place_info_dict.items():
        for syn in info["synonyms"]:
            if syn and re.search(re.escape(syn), text, flags=re.IGNORECASE):
                return standard
    return None

# -----------------------------
# 3) 감정 분류 (Rule 기반)
# -----------------------------
def load_sentiment_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def classify_sentiment_rule(text: str, sentiment_dict: dict) -> str | None:
    for category, keywords in sentiment_dict.items():
        if any(kw in text for kw in keywords):
            return category
    return None

# -----------------------------
# 4) LLM 보완 (Gemini)
# -----------------------------
def classify_with_llm(text: str, place_info_dict: dict, sentiment_categories: list[str]) -> tuple[str, str]:
    prompt = f"""
    다음 텍스트에서 언급된 매장명을 표준 매장명으로 추출하고,
    감정은 아래 카테고리 중 가장 적합한 하나를 선택해줘.

    텍스트: {text}

    매장 후보 (예시): {list(place_info_dict.keys())[:20]}

    감정 카테고리 후보:
    {sentiment_categories}

    출력 형식:
    매장명: <표준매장명 또는 None>
    감정: <카테고리>
    """

    response = gemini_model.generate_content(prompt)
    answer = response.text.strip()

    place, sentiment = None, "기타_중립"
    for line in answer.splitlines():
        if line.startswith("매장명:"):
            place = line.replace("매장명:", "").strip()
        if line.startswith("감정:"):
            sentiment = line.replace("감정:", "").strip()

    return place, sentiment

# -----------------------------
# 5) 실행 부분 (Hybrid 적용)
# -----------------------------
def main():
    CLEAN_FILE = "data/cafe_posts_clean.csv"
    PLACE_INFO_FILE = "data/jewelry_place_info_dict.json"
    SENTIMENT_DICT_FILE = "data/place_sentiment_keywords_by_category.json"

    df = load_and_sample(CLEAN_FILE, n=50, random_state=42)
    place_info_dict = load_place_info_dict(PLACE_INFO_FILE)
    sentiment_dict = load_sentiment_dict(SENTIMENT_DICT_FILE)
    sentiment_categories = list(sentiment_dict.keys())

    results = []
    for _, row in df.iterrows():
        text = f"{row.get('title','')} {row.get('text','')} {row.get('comments','')}"

        # 1) Rule 기반 먼저 시도
        place = normalize_place_rule(text, place_info_dict)
        sentiment = classify_sentiment_rule(text, sentiment_dict)

        # 2) Rule 실패 시 LLM 호출
        if not place or not sentiment:
            place_llm, sentiment_llm = classify_with_llm(text, place_info_dict, sentiment_categories)
            place = place or place_llm
            sentiment = sentiment or sentiment_llm

        if place:
            info = place_info_dict.get(place, {})
            record = {
                "article_id": row.get("naver_article_id", None),
                "place_name": place,
                "pbrand_name": info.get("pbrand_name"),
                "location_lv1": info.get("location_lv1"),
                "location_lv2": info.get("location_lv2"),
                "sentiment": sentiment
            }

            # 모두 None 또는 "None"이면 제외
            if not all((record.get(k) is None or record.get(k) == "None") for k in ["place_name", "pbrand_name", "location_lv1", "location_lv2"]):
                results.append(record)

    results_df = pd.DataFrame(results)
    # print("\n=== 라벨링 결과 미리보기 (상위 5행) ===")
    # print(results_df.head(5))
    
    # ✅ 전체 결과 CSV 저장
    OUTPUT_FILE = "data/labeled_sample_sentiment.csv"
    save_csv(results_df, OUTPUT_FILE)

   
if __name__ == "__main__":
    main()


전체 결과인데 행이 15개 뿐? -> 정확도 대략 50% 나옴
샘플 load_and_sample 함수 내 n=50 제거하고 전체 데이터 대상. 
하니까 너무 오래걸림.40분 됐는데 아직도 안끝남;; -> 97분에서 스탑. 

In [ ]:
import pandas as pd
import os
import re
import json
from dotenv import load_dotenv
import google.generativeai as genai  # Gemini SDK

# ------------------------
# ① 환경설정 & API 키 불러오기
# ------------------------
load_dotenv()
google_key = os.getenv("GOOGLE_API_KEY")
if not google_key:
    raise ValueError("환경변수 GOOGLE_API_KEY가 없습니다. .env 파일 확인!")

# ------------------------
# ② Gemini 클라이언트 초기화
# ------------------------
genai.configure(api_key=google_key)
gemini_model = genai.GenerativeModel("gemini-1.5-flash")

# ------------------------
# CSV 저장 헬퍼 함수
# ------------------------
def save_csv(df: pd.DataFrame, path: str):
    """DataFrame을 안전하게 CSV로 저장"""
    df.to_csv(path, index=False, encoding="utf-8-sig")
    print(f" CSV 저장 완료: {path}")

# -----------------------------
# 1) 데이터 불러오기 (✅전체 데이터 사용)
# -----------------------------
def load_data(filepath: str) -> pd.DataFrame:
    """CSV 파일 전체 불러오기"""
    return pd.read_csv(filepath)

# -----------------------------
# 2) 매장명 정규화 (Rule 기반)
# -----------------------------
def load_place_info_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def normalize_place_rule(text: str, place_info_dict: dict) -> str | None:
    for standard, info in place_info_dict.items():
        for syn in info["synonyms"]:
            if syn and re.search(re.escape(syn), text, flags=re.IGNORECASE):
                return standard
    return None

# -----------------------------
# 3) 감정 분류 (Rule 기반)
# -----------------------------
def load_sentiment_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def classify_sentiment_rule(text: str, sentiment_dict: dict) -> str | None:
    for category, keywords in sentiment_dict.items():
        if any(kw in text for kw in keywords):
            return category
    return None

# -----------------------------
# 4) LLM 보완 (Gemini)
# -----------------------------
def classify_with_llm(text: str, place_info_dict: dict, sentiment_categories: list[str]) -> tuple[str, str]:
    prompt = f"""
    다음 텍스트에서 언급된 매장명을 표준 매장명으로 추출하고,
    감정은 아래 카테고리 중 가장 적합한 하나를 선택해줘.

    텍스트: {text}

    매장 후보 (예시): {list(place_info_dict.keys())[:20]}

    감정 카테고리 후보:
    {sentiment_categories}

    출력 형식:
    매장명: <표준매장명 또는 None>
    감정: <카테고리>
    """

    response = gemini_model.generate_content(prompt)
    answer = response.text.strip()

    place, sentiment = None, "기타_중립"
    for line in answer.splitlines():
        if line.startswith("매장명:"):
            place = line.replace("매장명:", "").strip()
        if line.startswith("감정:"):
            sentiment = line.replace("감정:", "").strip()

    return place, sentiment

# -----------------------------
# 5) 실행 부분 (Hybrid 적용)
# -----------------------------
def main():
    CLEAN_FILE = "data/cafe_posts_clean.csv"
    PLACE_INFO_FILE = "data/jewelry_place_info_dict.json"
    SENTIMENT_DICT_FILE = "data/place_sentiment_keywords_by_category.json"

    # ✅ 샘플링 제거 → 전체 데이터 사용
    df = load_data(CLEAN_FILE)

    place_info_dict = load_place_info_dict(PLACE_INFO_FILE)
    sentiment_dict = load_sentiment_dict(SENTIMENT_DICT_FILE)
    sentiment_categories = list(sentiment_dict.keys())

    results = []
    for _, row in df.iterrows():
        text = f"{row.get('title','')} {row.get('text','')} {row.get('comments','')}"

        # 1) Rule 기반 먼저 시도
        place = normalize_place_rule(text, place_info_dict)
        sentiment = classify_sentiment_rule(text, sentiment_dict)

        # 2) Rule 실패 시 LLM 호출
        if not place or not sentiment:
            place_llm, sentiment_llm = classify_with_llm(text, place_info_dict, sentiment_categories)
            place = place or place_llm
            sentiment = sentiment or sentiment_llm

        if place:
            info = place_info_dict.get(place, {})
            record = {
                "article_id": row.get("naver_article_id", None),
                "place_name": place,
                "pbrand_name": info.get("pbrand_name"),
                "location_lv1": info.get("location_lv1"),
                "location_lv2": info.get("location_lv2"),
                "sentiment": sentiment
            }

            # 모두 None 또는 "None"이면 제외
            if not all((record.get(k) is None or record.get(k) == "None") for k in ["place_name", "pbrand_name", "location_lv1", "location_lv2"]):
                results.append(record)

    results_df = pd.DataFrame(results)
    
    # ✅ 전체 결과 CSV 저장
    OUTPUT_FILE = "data/labeled_sentiment_all.csv"
    save_csv(results_df, OUTPUT_FILE)

if __name__ == "__main__":
    main()


sentiment2.csv : 불러오기 n=100, random state=42 -> 15행, 정확도 약 50%
sentiment3.csv : 불러오기 n=100, random state 제거 -> 11행, 정확도 약 15%
sentiment4.csv : 불러오기 n=50, main n=50, random state 제거 -> 11행, 정확도 약 15%

In [ ]:
import pandas as pd
import os
import re
import json
from dotenv import load_dotenv
import google.generativeai as genai  # Gemini SDK

# ------------------------
# ① 환경설정 & API 키 불러오기
# ------------------------
load_dotenv()
google_key = os.getenv("GOOGLE_API_KEY")
if not google_key:
    raise ValueError("환경변수 GOOGLE_API_KEY가 없습니다. .env 파일 확인!")

# ------------------------
# ② Gemini 클라이언트 초기화
# ------------------------
genai.configure(api_key=google_key)
gemini_model = genai.GenerativeModel("gemini-1.5-flash")

# ------------------------
# ✅ CSV 저장 헬퍼 함수
# ------------------------
def save_csv(df: pd.DataFrame, path: str):
    """DataFrame을 안전하게 CSV로 저장"""
    df.to_csv(path, index=False, encoding="utf-8-sig")
    print(f"✅ CSV 저장 완료: {path}")

# -----------------------------
# 1) 데이터 불러오기 + 샘플링
# -----------------------------
def load_and_sample(filepath: str, n=100, random_state=42) -> pd.DataFrame:
    df = pd.read_csv(filepath)
    sample_df = df.sample(n=n, random_state=random_state)
    return sample_df.reset_index(drop=True)

# -----------------------------
# 2) 매장명 정규화 (Rule 기반)
# -----------------------------
def load_place_info_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def normalize_place_rule(text: str, place_info_dict: dict) -> str | None:
    for standard, info in place_info_dict.items():
        for syn in info["synonyms"]:
            if syn and re.search(re.escape(syn), text, flags=re.IGNORECASE):
                return standard
    return None

# -----------------------------
# 3) 감정 분류 (Rule 기반)
# -----------------------------
def load_sentiment_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def classify_sentiment_rule(text: str, sentiment_dict: dict) -> str | None:
    for category, keywords in sentiment_dict.items():
        if any(kw in text for kw in keywords):
            return category
    return None

# -----------------------------
# 4) LLM 보완 (Gemini)
# -----------------------------
def classify_with_llm(text: str, place_info_dict: dict, sentiment_categories: list[str]) -> tuple[str, str]:
    prompt = f"""
    다음 텍스트에서 언급된 매장명을 표준 매장명으로 추출하고,
    감정은 아래 카테고리 중 가장 적합한 하나를 선택해줘.

    텍스트: {text}

    매장 후보 (예시): {list(place_info_dict.keys())[:20]}

    감정 카테고리 후보:
    {sentiment_categories}

    출력 형식:
    매장명: <표준매장명 또는 None>
    감정: <카테고리>
    출력은 반드시 JSON 형식으로만 했으면 해.
    예: {"매장명": "현대백화점 무역센터점", "감정": "인테리어_긍정"}
    """

    response = gemini_model.generate_content(prompt)
    answer = response.text.strip()

    place, sentiment = None, "기타_중립"
    for line in answer.splitlines():
        if line.startswith("매장명:"):
            place = line.replace("매장명:", "").strip()
        if line.startswith("감정:"):
            sentiment = line.replace("감정:", "").strip()

    return place, sentiment

# -----------------------------
# 5) 실행 부분 (Hybrid 적용)
# -----------------------------
def main():
    CLEAN_FILE = "data/cafe_posts_clean.csv"
    PLACE_INFO_FILE = "data/jewelry_place_info_dict.json"
    SENTIMENT_DICT_FILE = "data/place_sentiment_keywords_by_category.json"

    df = load_and_sample(CLEAN_FILE, n=50, random_state=42)
    place_info_dict = load_place_info_dict(PLACE_INFO_FILE)
    sentiment_dict = load_sentiment_dict(SENTIMENT_DICT_FILE)
    sentiment_categories = list(sentiment_dict.keys())

    results = []
    for _, row in df.iterrows():
        text = f"{row.get('title','')} {row.get('text','')} {row.get('comments','')}"

        # 1) Rule 기반 먼저 시도
        place = normalize_place_rule(text, place_info_dict)
        sentiment = classify_sentiment_rule(text, sentiment_dict)

        # 2) Rule 실패 시 LLM 호출
        if not place or not sentiment:
            place_llm, sentiment_llm = classify_with_llm(text, place_info_dict, sentiment_categories)
            place = place or place_llm
            sentiment = sentiment or sentiment_llm

        if place:
            info = place_info_dict.get(place, {})
            record = {
                "article_id": row.get("naver_article_id", None),
                "place_name": place,
                "pbrand_name": info.get("pbrand_name"),
                "location_lv1": info.get("location_lv1"),
                "location_lv2": info.get("location_lv2"),
                "sentiment": sentiment
            }

            # 모두 None 또는 "None"이면 제외
            if not all((record.get(k) is None or record.get(k) == "None") for k in ["place_name", "pbrand_name", "location_lv1", "location_lv2"]):
                results.append(record)

    results_df = pd.DataFrame(results)
    # print("\n=== 라벨링 결과 미리보기 (상위 5행) ===")
    # print(results_df.head(5))
    
    # ✅ 결과 CSV 저장
    OUTPUT_FILE = "data/labeled_sample_sentiment2.csv"
    save_csv(results_df, OUTPUT_FILE)

   
if __name__ == "__main__":
    main()


전체 행 대상으로 하되 변경사항 
1. if not place or not sentiment 조건을 AND 조건으로 변경 -> Gemini 호출횟수 완화
2. 얼마나 진행됐는지 진행률 확인 코드 추가
3. 일단 200행만 돌려서 평균 처리 속도 측정 

라벨링 진행중: 5%, 617/11969, 8m30s -> 이대로면 완료까지 3시간 예상. 스탑.

In [ ]:
import pandas as pd
import os
import re
import json
import time
from dotenv import load_dotenv
import google.generativeai as genai  # Gemini SDK
from tqdm import tqdm   # ✅ 진행률 표시

# ------------------------
# ① 환경설정 & API 키 불러오기
# ------------------------
load_dotenv()
google_key = os.getenv("GOOGLE_API_KEY")
if not google_key:
    raise ValueError("환경변수 GOOGLE_API_KEY가 없습니다. .env 파일 확인!")

# ------------------------
# ② Gemini 클라이언트 초기화
# ------------------------
genai.configure(api_key=google_key)
gemini_model = genai.GenerativeModel("gemini-1.5-flash")

# ------------------------
# ✅ CSV 저장 헬퍼 함수
# ------------------------
def save_csv(df: pd.DataFrame, path: str):
    """DataFrame을 안전하게 CSV로 저장"""
    df.to_csv(path, index=False, encoding="utf-8-sig")
    print(f"✅ CSV 저장 완료: {path}")

# -----------------------------
# 1) 데이터 불러오기 (전체 데이터 사용)
# -----------------------------
def load_data(filepath: str) -> pd.DataFrame:
    """CSV 파일 전체 불러오기"""
    return pd.read_csv(filepath)

# -----------------------------
# 2) 매장명 정규화 (Rule 기반)
# -----------------------------
def load_place_info_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def normalize_place_rule(text: str, place_info_dict: dict) -> str | None:
    for standard, info in place_info_dict.items():
        for syn in info["synonyms"]:
            if syn and re.search(re.escape(syn), text, flags=re.IGNORECASE):
                return standard
    return None

# -----------------------------
# 3) 감정 분류 (Rule 기반)
# -----------------------------
def load_sentiment_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def classify_sentiment_rule(text: str, sentiment_dict: dict) -> str | None:
    for category, keywords in sentiment_dict.items():
        if any(kw in text for kw in keywords):
            return category
    return None

# -----------------------------
# 4) LLM 보완 (Gemini)
# -----------------------------
def classify_with_llm(text: str, place_info_dict: dict, sentiment_categories: list[str]) -> tuple[str, str]:
    prompt = f"""
    다음 텍스트에서 언급된 매장명을 표준 매장명으로 추출하고,
    감정은 아래 카테고리 중 가장 적합한 하나를 선택해줘.

    텍스트: {text}

    매장 후보 (예시): {list(place_info_dict.keys())[:20]}

    감정 카테고리 후보:
    {sentiment_categories}

    출력 형식:
    매장명: <표준매장명 또는 None>
    감정: <카테고리>
    """

    response = gemini_model.generate_content(prompt)
    answer = response.text.strip()

    place, sentiment = None, "기타_중립"
    for line in answer.splitlines():
        if line.startswith("매장명:"):
            place = line.replace("매장명:", "").strip()
        if line.startswith("감정:"):
            sentiment = line.replace("감정:", "").strip()

    return place, sentiment

# -----------------------------
# 5) 실행 부분 (Hybrid 적용)
# -----------------------------
def main():
    CLEAN_FILE = "data/cafe_posts_clean.csv"
    PLACE_INFO_FILE = "data/jewelry_place_info_dict.json"
    SENTIMENT_DICT_FILE = "data/place_sentiment_keywords_by_category.json"

    df = load_data(CLEAN_FILE)
    place_info_dict = load_place_info_dict(PLACE_INFO_FILE)
    sentiment_dict = load_sentiment_dict(SENTIMENT_DICT_FILE)
    sentiment_categories = list(sentiment_dict.keys())

    results = []

    start = time.time()

    # ✅ tqdm으로 진행률 표시
    for _, row in tqdm(df.iterrows(), total=len(df), desc="라벨링 진행중"):
        text = f"{row.get('title','')} {row.get('text','')} {row.get('comments','')}"

        # 1) Rule 기반 먼저 시도
        place = normalize_place_rule(text, place_info_dict)
        sentiment = classify_sentiment_rule(text, sentiment_dict)

        # ✅ 조건 완화: 매장명 & 감정 둘 다 실패했을 때만 LLM 호출
        if not place and not sentiment:
            place_llm, sentiment_llm = classify_with_llm(text, place_info_dict, sentiment_categories)
            place = place or place_llm
            sentiment = sentiment or sentiment_llm

        if place:
            info = place_info_dict.get(place, {})
            record = {
                "article_id": row.get("naver_article_id", None),
                "place_name": place,
                "pbrand_name": info.get("pbrand_name"),
                "location_lv1": info.get("location_lv1"),
                "location_lv2": info.get("location_lv2"),
                "sentiment": sentiment
            }

            # 모두 None 또는 "None"이면 제외
            if not all((record.get(k) is None or record.get(k) == "None")
                       for k in ["place_name", "pbrand_name", "location_lv1", "location_lv2"]):
                results.append(record)

    results_df = pd.DataFrame(results)

    # ✅ 전체 결과 CSV 저장
    OUTPUT_FILE = "data/labeled_sentiment_all.csv"
    save_csv(results_df, OUTPUT_FILE)

    elapsed = time.time() - start
    print(f"\n⏱ 전체 처리 시간: {elapsed/60:.2f}분")
    print(f"📊 최종 결과 행 수: {results_df.shape[0]}")

if __name__ == "__main__":
    main()


In [ ]:
라벨링 진행중: 5%, 617/11969, 8m30s